Train ViT model

In [4]:
# import os
# import av
# import torch
# import numpy as np
# import random
# import json
# import time
# import subprocess
# import psutil
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from transformers import ViTForImageClassification, ViTImageProcessor
# from collections import defaultdict
# from torch.utils.data import DataLoader, Dataset

# # 定义配置
# config = {
#     "model_name": "google/vit-base-patch16-224-in21k",
#     "image_processor_name": "google/vit-base-patch16-224-in21k",
#     "num_samples": 10000,
#     "num_classes": 400,  # For flexible dataset input
#     "num_samples_per_class": 25,  # For flexible dataset input
#     "video_list_path": "archive/kinetics400_val_list_videos.txt",
#     "video_directory": "archive/zoom_blur",
#     "use_exact": True
# }

# # 加载预训练的ViT模型和处理器
# model = ViTForImageClassification.from_pretrained(config["model_name"], num_labels=400)
# image_processor = ViTImageProcessor.from_pretrained(config["image_processor_name"])

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.to(device)

# # 读取视频文件和标签
# video_labels = defaultdict(list)
# with open(config["video_list_path"], "r") as f:
#     for line in f:
#         name, label = line.strip().split()
#         video_labels[int(label)].append(name)

# # 准备样本
# sampled_files = []
# true_labels = []
# selected_classes = random.sample(list(video_labels.keys()), config["num_classes"])
# for cls in selected_classes:
#     sampled_files.extend(random.sample(video_labels[cls], config["num_samples_per_class"]))
#     true_labels.extend([cls] * config["num_samples_per_class"])

# # 自定义数据集类
# class Kinetics400Dataset(Dataset):
#     def __init__(self, video_files, labels, image_processor):
#         self.video_files = video_files
#         self.labels = labels
#         self.image_processor = image_processor

#     def __len__(self):
#         return len(self.video_files)

#     def __getitem__(self, idx):
#         video_file = self.video_files[idx]
#         label = self.labels[idx]
#         container = av.open(os.path.join(config["video_directory"], video_file))
#         frame = container.decode(video=0).__next__().to_image()  # 获取第一帧
#         inputs = self.image_processor(images=frame, return_tensors="pt")
#         inputs = {k: v.squeeze(0) for k, v in inputs.items()}
#         return inputs['pixel_values'], torch.tensor(label)

# # 准备数据集和数据加载器
# dataset = Kinetics400Dataset(sampled_files, true_labels, image_processor)
# dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# # 微调模型
# optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
# criterion = torch.nn.CrossEntropyLoss()

# model.train()
# for epoch in range(5):  # 你可以根据需要调整epoch数量
#     for batch in dataloader:
#         images, labels = batch
#         images = images.to(device)
#         labels = labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(images).logits
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# # 预测函数
# model.eval()
# def predict_video(video_file):
#     container = av.open(os.path.join(config["video_directory"], video_file))
#     frame = container.decode(video=0).__next__().to_image()  # 获取第一帧
#     inputs = image_processor(images=frame, return_tensors="pt")
#     inputs = {k: v.to(device) for k, v in inputs.items()}

#     with torch.no_grad():
#         outputs = model(**inputs).logits
#         predicted_label = torch.argmax(outputs, dim=-1).item()
#     return predicted_label

# # 对样本进行预测并评估
# predictions = [predict_video(video_file) for video_file in sampled_files]

# accuracy = accuracy_score(true_labels, predictions)
# precision = precision_score(true_labels, predictions, average='weighted')
# recall = recall_score(true_labels, predictions, average='weighted')
# f1 = f1_score(true_labels, predictions, average='weighted')

# print(f"Accuracy: {accuracy:.4f}")
# print(f"Precision: {precision:.4f}")
# print(f"Recall: {recall:.4f}")
# print(f"F1 Score: {f1:.4f}")


/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Loss: 5.392894744873047
Epoch 2, Loss: 4.374405860900879
Epoch 3, Loss: 2.4989871978759766
Epoch 4, Loss: 2.265422821044922
Epoch 5, Loss: 0.9371886253356934
Accuracy: 0.8917
Precision: 0.9100
Recall: 0.8917
F1 Score: 0.8870


Train ViT model and save the finetune results ONLY work on the first frame

In [18]:
import os
import av
import torch
import numpy as np
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import ViTForImageClassification, ViTImageProcessor
from collections import defaultdict
from torch.utils.data import DataLoader, Dataset

# 定义配置
config = {
    "model_name": "google/vit-base-patch16-224-in21k",
    "image_processor_name": "google/vit-base-patch16-224-in21k",
    "num_samples": 10000,
    "num_classes": 400,  # For flexible dataset input
    "num_samples_per_class": 25,  # For flexible dataset input
    "video_list_path": "archive/kinetics400_val_list_videos.txt",
    "video_directory": "archive/videos_val",
    "use_exact": True
}

# 加载预训练的ViT模型和处理器
model = ViTForImageClassification.from_pretrained(config["model_name"], num_labels=400)
image_processor = ViTImageProcessor.from_pretrained(config["image_processor_name"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 读取视频文件和标签
video_labels = defaultdict(list)
with open(config["video_list_path"], "r") as f:
    for line in f:
        name, label = line.strip().split()
        video_labels[int(label)].append(name)

# 准备样本
sampled_files = []
true_labels = []
selected_classes = random.sample(list(video_labels.keys()), config["num_classes"])
for cls in selected_classes:
    sampled_files.extend(random.sample(video_labels[cls], config["num_samples_per_class"]))
    true_labels.extend([cls] * config["num_samples_per_class"])

# 自定义数据集类
class Kinetics400Dataset(Dataset):
    def __init__(self, video_files, labels, image_processor):
        self.video_files = video_files
        self.labels = labels
        self.image_processor = image_processor

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_file = self.video_files[idx]
        label = self.labels[idx]
        container = av.open(os.path.join(config["video_directory"], video_file))
        frame = container.decode(video=0).__next__().to_image()  # 获取第一帧
        inputs = self.image_processor(images=frame, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs['pixel_values'], torch.tensor(label)

# 准备数据集和数据加载器
dataset = Kinetics400Dataset(sampled_files, true_labels, image_processor)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 微调模型
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(5):  # 你可以根据需要调整epoch数量
    for batch in dataloader:
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# 保存微调后的模型
model_save_path = "finetuned_vit_model5.pth"
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

# 定义推理函数
def predict_video(video_file, model, image_processor):
    container = av.open(video_file)
    frame = container.decode(video=0).__next__().to_image()  # 获取第一帧
    inputs = image_processor(images=frame, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs).logits
        predicted_label = torch.argmax(outputs, dim=-1).item()
        probabilities = torch.nn.functional.softmax(outputs, dim=-1)
        predicted_score = probabilities[0, predicted_label].item()

    return predicted_label, predicted_score



Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 

work on 5 frame for each video, finetune

In [12]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

config = {
    "model_name": "google/vit-base-patch16-224-in21k",
    "image_processor_name": "google/vit-base-patch16-224-in21k",
    "num_samples": 10000,
    "num_classes": 400,
    "num_samples_per_class": 25,
    "video_list_path": "archive/kinetics400_val_list_videos.txt",
    "video_directory": "archive/videos_val",
    "use_exact": True,
    "frames_per_video": 25,
    "batch_size": 64,
    "num_epochs": 100,
    "learning_rate": 1e-4,
    "patience": 5  # 用于早停
}

# 数据分割
train_files, val_files, train_labels, val_labels = train_test_split(
    train_files, train_labels, test_size=0.2, stratify=train_labels, random_state=42
)

train_dataset = Kinetics400Dataset(train_files, train_labels, image_processor, config["frames_per_video"])
val_dataset = Kinetics400Dataset(val_files, val_labels, image_processor, config["frames_per_video"])
train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=config["batch_size"])

def train_model(model, train_dataloader, val_dataloader, num_epochs, patience):
    optimizer = torch.optim.AdamW(model.parameters(), lr=config["learning_rate"])
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3)
    criterion = torch.nn.CrossEntropyLoss()

    best_val_accuracy = 0
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Average Loss: {avg_loss:.4f}")

        # Validation
        val_accuracy = evaluate_model(model, val_dataloader)
        print(f"Validation Accuracy: {val_accuracy:.4f}")

        scheduler.step(val_accuracy)

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            epochs_no_improve = 0
            torch.save(model.state_dict(), "finetuned_vit_model_20.pth")
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print("Early stopping!")
                break

    model.load_state_dict(torch.load("finetuned_vit_model_20.pth"))
    return model

# 修改评估函数以支持数据加载器
def evaluate_model(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

# 训练模型
model = train_model(model, train_dataloader, val_dataloader, config["num_epochs"], config["patience"])

# 在测试集上评估
test_dataset = Kinetics400Dataset(test_files, test_labels, image_processor, config["frames_per_video"])
test_dataloader = DataLoader(test_dataset, batch_size=config["batch_size"])
test_accuracy = evaluate_model(model, test_dataloader)
print(f"Test Accuracy: {test_accuracy:.4f}")

NameError: name 'train_files' is not defined

# 重新加载模型进行推理

In [24]:
import os
import av
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import ViTForImageClassification, ViTImageProcessor
from torch.utils.data import DataLoader, Dataset

# Reuse the config from the training script
config = {
    "model_name": "google/vit-base-patch16-224-in21k",
    "image_processor_name": "google/vit-base-patch16-224-in21k",
    "num_samples": 2000,
    "num_classes": 400,
    "num_samples_per_class": 5,
    "video_list_path": "archive/kinetics400_val_list_videos.txt",
    "video_directory": "archive/videos_val",
    "use_exact": True
}

# Load the fine-tuned model
model = ViTForImageClassification.from_pretrained(config["model_name"], num_labels=400)
model.load_state_dict(torch.load("finetuned_vit_model_20.pth"))
image_processor = ViTImageProcessor.from_pretrained(config["image_processor_name"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Modified Kinetics400Dataset to extract middle frame
class Kinetics400Dataset(Dataset):
    def __init__(self, video_files, labels, image_processor):
        self.video_files = video_files
        self.labels = labels
        self.image_processor = image_processor
    
    def __len__(self):
        return len(self.video_files)
    
    def __getitem__(self, idx):
        video_file = self.video_files[idx]
        label = self.labels[idx]
        
        container = av.open(os.path.join(config["video_directory"], video_file))
        video = container.streams.video[0]
        
        # Get the middle frame
        middle_frame_index = video.frames // 2
        for i, frame in enumerate(container.decode(video=0)):
            if i == middle_frame_index:
                middle_frame = frame.to_image()
                break
        
        inputs = self.image_processor(images=middle_frame, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs['pixel_values'], torch.tensor(label)

# Prepare the evaluation dataset
video_labels = {}
with open(config["video_list_path"], "r") as f:
    for line in f:
        name, label = line.strip().split()
        video_labels[name] = int(label)

eval_files = list(video_labels.keys())
eval_labels = [video_labels[file] for file in eval_files]

eval_dataset = Kinetics400Dataset(eval_files, eval_labels, image_processor)
eval_dataloader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

# Evaluation
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_dataloader:
        images, labels = batch
        images = images.to(device)
        
        outputs = model(images).logits
        preds = torch.argmax(outputs, dim=-1).cpu().numpy()
        
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')
recall = recall_score(all_labels, all_preds, average='weighted')
f1 = f1_score(all_labels, all_preds, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
deprecated pixel format used, make sure you did set range correctly
deprecated pixel format used, make sure you did set range correctly


Accuracy: 0.4901
Precision: 0.5238
Recall: 0.4901
F1 Score: 0.4946
